In [1]:
import mlflow
import os

mlflow.set_tracking_uri("http://localhost:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'mlflow'  
os.environ['AWS_SECRET_ACCESS_KEY'] = 'mlflowpass' 

os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:10000'

In [2]:
unet = mlflow.pytorch.load_model("runs:/d1a022812dcd4b0a99d693a7920d38e5/model")

/home/kmj388/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

2025/08/06 08:32:21 WARNING mlflow.pytorch: Stored model version '2.1.0' does not match installed PyTorch version '2.7.1+cu126'

2025/08/06 08:32:21 WARNING mlflow.pytorch: Stored model version '2.1.0' does not match installed PyTorch version '2.7.1+cu126'


ModuleNotFoundError: No module named 'unet'